### Data Gathering

Gather all requirements.txt files from repositories having python as language. 

In [ ]:
from google.cloud import bigquery

job = bigquery.job.QueryJobConfig()
job.use_legacy_sql = True
client = bigquery.Client(default_query_job_config=job)


query = """
 SELECT D.id AS id,
       repo_name,
       path,
       content
FROM   (SELECT id,
               content
        FROM   [bigquery-public-data.github_repos.contents]
        GROUP  BY id,
                  content) AS D
       INNER JOIN (SELECT id,
                          C.repo_name AS repo_name,
                          path
                   FROM   (SELECT id,
                                  repo_name,
                                  path
                           FROM
                  [bigquery-public-data:github_repos.files]
                           WHERE  LOWER(path) LIKE '%requirements.txt'
                           GROUP  BY path,
                                     id,
                                     repo_name) AS C
                          INNER JOIN (SELECT repo_name,
                                             language.name
                                      FROM
                          [bigquery-public-data.github_repos.languages]
                                      WHERE  LOWER(language.name) LIKE
                                             '%python%'
                                      GROUP  BY language.name,
                                                repo_name) AS F
                                  ON C.repo_name = F.repo_name) AS E
               ON E.id = D.id  
"""

In [ ]:
query_job = client.query(query)

#### Store the collected data in json format

In [ ]:
content_list = []
for row in query_job:
    content = {
        "id": row.get('id'),
        "repo_name": row.get('repo_name'),
        "path": row.get("path"),
        "content": row.get('content')
    }
    content_list.append(content)

In [ ]:
import json

In [ ]:
with open('python-bigquery-data.json', 'w') as f:
    json.dump(content_list, f)

#### Take out all requirements.txt contents out of the stored json and dump them in a single requirements.txt file

In [ ]:
with open('requirements.txt', 'a') as r, open('python-bigquery-data.json', 'r')  as f:
    content = json.load(f)
    for x in content:
        if x.get('content'):
            r.write(x.get('content'))

### Data Filtering

This is the trickiest part. We do the following:

- We iterate through every requirements.txt file and dump the contents in a temp file.
- After that, we check if it's a valid requirements.txt file.
- Then we check if the package name is a valid package name after querying it on pypi.

We create a json file to store all the valid manifests.

In [ ]:
from pip._internal.req.req_file import parse_requirements
from pip._internal.download import PipSession
import requests
import requests_cache
from datetime import timedelta

In [ ]:
from pip._vendor.distlib.util import normalize_name

In [ ]:
requests_cache.install_cache(expire_after=timedelta(hours=5))

In [ ]:
manifest_json = [{
    "ecosystem": "pypi",
    "package_list": []
}]
session = PipSession()

with open("python-bigquery-data.json", "r") as f, open('error-log-pip.txt', 'a') as log:
    content = json.load(f)
    for x in content:
        if x.get('content'):
            with open("temp-requirements.txt", "w") as w:
                w.write(x.get('content'))
            req_names = []
            try:
                for p in parse_requirements("temp-requirements.txt", session=session):
                    if p.name:
                        name = normalize_name(p.name)
                        r = requests.get(url='https://pypi.org/pypi/{}/json'.format(name), headers={"Accept": "application/json"})
                        if r.status_code == 200:
                            print("Package: {} done".format(name))
                            req_names.append(name)
            except Exception as e:
                log.write(e)
            manifest_json[0].get("package_list").append(req_names)
                    
# for p in parse_requirements("requirements-final.txt", session=PipSession()):
#     print(p.name)

In [ ]:
manifest_json[0].get("package_list")

In [ ]:
with open("manifest-list-with-pip-api-normalized.json", "w") as w:
    json.dump(manifest_json, w)

#### We filter the data further by keeping only manifests with length > 0 and removing duplicate manifests.

In [ ]:
manifest_json = [{
    "ecosystem": "pypi",
    "package_list": []
}]

with open('manifest-list-with-pip-api-normalized.json', 'r') as f:
    content = json.load(f)
    manifest_set = set()
    for package_list in content[0].get('package_list'):
        if len(package_list) > 0:
            manifest_set.add(frozenset(package_list))

In [ ]:
manifest_json[0]['package_list'] = [list(x) for x in manifest_set]

In [ ]:
with open('manifest-list-trimmed-unique.json', 'w') as w:
    json.dump(manifest_json, w)